In [1]:
# Instalación de paquetes requeridos
#%pip install unidecode
#%pip install pandas

In [2]:
# Importar paquetes requeridos
import pandas as pd
from unidecode import unidecode
from re import sub

In [3]:
# Definición de función que permite dejar los nombres de los cursos en un formato adecuado
def format_text(input_string):
    no_a = unidecode(str(input_string)) # Eliminar tildes
    no_sc = sub('[^a-zA-Z0-9]+', '', no_a) # Eliminar caracteres especiales
    return no_sc.lower().strip().replace(' ', '') # Eliminar espacios y pasar todo a minúscula

# Definición de función que permite dejar las siglas de los cursos en un formato adecuado
def format_course_number(input_string):
    if len(input_string) == 1: # Si el número de curso tiene 1 dígito. Ej. COM1
        return f'00{input_string}' # Agrega dos ceros. Ej. COM001

    elif len(input_string) == 2: # Si el número de curso tiene 2 dígitos. Ej. COM01
        return f'0{input_string}' # Agrega un cero. Ej. COM001
    return input_string

**Especificar a continuación los nombres de los archivos conteniendo el catálogo cognos y el catálogo VRAI junto con las respuestas del formulario**

In [4]:
df_courses_global_file = 'Cursos programados 20261.xlsx'
df_courses_vrai_file = 'formulario_limpio_rezagados.xlsx'

A continuación se procede a importar y formatear la información de los cursos desde el **catálogo cognos** (que contiene todos los cursos a dictar)

In [5]:
cols_courses_global = ['NRC', 'Escuela', 'Materia', 'Número Curso', 'Nombre Curso']
df_courses_global = pd.read_excel(df_courses_global_file, dtype=str)[cols_courses_global].astype(str)

df_courses_global[cols_courses_global] = df_courses_global[cols_courses_global].apply(lambda x : x.str.strip())
df_courses_global['Número Curso'] = df_courses_global['Número Curso'].apply(format_course_number)
df_courses_global['Sigla'] = df_courses_global['Materia'] + df_courses_global['Número Curso']
df_courses_global = df_courses_global.drop(['Materia', 'Número Curso'], axis=1)
df_courses_global['Nombre Curso (formateado)'] = df_courses_global['Nombre Curso'].apply(format_text)
df_courses_global['Sigla (formateado)'] = df_courses_global['Sigla'].apply(format_text)

In [6]:
df_courses_global

,NRC,Escuela,Nombre Curso,Sigla,Nombre Curso (formateado),Sigla (formateado)
0,30729,94 - Arquitectura,Tesis de Doctorado II,ADU4019,tesisdedoctoradoii,adu4019
1,27568,94 - Arquitectura,Tesis de Doctorado III,ADU4020,tesisdedoctoradoiii,adu4020
2,20077,94 - Arquitectura,Proyecto de Tesis Doctoral,ADU4101,proyectodetesisdoctoral,adu4101
3,20078,94 - Arquitectura,Proyecto de Tesis Doctoral,ADU4101,proyectodetesisdoctoral,adu4101
4,20606,94 - Arquitectura,Proyecto de Tesis Doctoral,ADU4101,proyectodetesisdoctoral,adu4101
...,...,...,...,...,...,...
7702,19763,56 - Historia,Seminario I Parte a,IHI301A,seminarioipartea,ihi301a
7703,19764,56 - Historia,Seminario I Parte a,IHI301A,seminarioipartea,ihi301a
7704,19765,56 - Historia,Seminario I Parte a,IHI301A,seminarioipartea,ihi301a
7705,19766,56 - Historia,Seminario I Parte a,IHI301A,seminarioipartea,ihi301a


A continuación se procede a importar y formatear la información de los cursos desde el **catálogo VRAI**

In [7]:
cols_courses_vrai = ['NRC', 'Escuela', 'Nombre Curso', 'Sigla']
df_courses_vrai = pd.read_excel(df_courses_vrai_file, sheet_name='Catálogo VRAI', dtype=str)[cols_courses_vrai].astype(str)

df_courses_vrai[cols_courses_vrai] = df_courses_vrai[cols_courses_vrai].apply(lambda x : x.str.strip())
#df_courses_vrai['Nombre Curso'] = df_courses_vrai['Nombre del curso']
df_courses_vrai['Nombre Curso (formateado)'] = df_courses_vrai['Nombre Curso'].apply(format_text)
df_courses_vrai['Sigla (formateado)'] = df_courses_vrai['Sigla'].apply(format_text)

A continuación se procede a importar y formatear la información de los **cursos generales que los estudiantes indicaron** en el formulario

In [8]:
cols_students = ['Nombre completo', 'RUT UC', 'Nombre Curso', 'Sigla', 'NRC']
df_students = pd.read_excel(df_courses_vrai_file, dtype=str, sheet_name="Formularios limpios")[cols_students].astype(str)

df_students[cols_students] = df_students[cols_students].apply(lambda x : x.str.strip())
df_students['Nombre Curso (formateado)'] = df_students['Nombre Curso'].apply(format_text)
df_students['Sigla (formateado)'] = df_students['Sigla'].apply(format_text)

A continuación se procede a importar y formatear la información de los **cursos deportivos que los estudiantes indicaron** en el formulario

In [34]:
cols_students_dpt = ['Nombre completo', 'RUT UC', 'Nombre Curso', 'Sigla', 'NRC']
df_students_dpt = pd.read_excel(df_courses_vrai_file, dtype=str, sheet_name='Cursos deportivos')[cols_students].astype(str)

df_students_dpt[cols_students_dpt] = df_students_dpt[cols_students_dpt].apply(lambda x : x.str.strip())
df_students_dpt['Nombre Curso (formateado)'] = df_students_dpt['Nombre Curso'].apply(format_text)
df_students_dpt['Sigla (formateado)'] = df_students_dpt['Sigla'].apply(format_text)

ValueError: Worksheet named 'Cursos deportivos' not found

A continuación se definen las funciones que se encargan de verificar NRC, sigla y nombre de los cursos

In [9]:
def verify_nrc(df, course_matches, selected_course_name_format, selected_nrc, obs):
    suggested_nrc = ''
    course_match_nrc = df.loc[(df['Nombre Curso (formateado)'] == selected_course_name_format) & (df['NRC'] == selected_nrc)]
    if not len(course_match_nrc):
        if len(course_matches) > 1:
            obs += 'ERROR, se requiere revisión. NRC no válido para nombre de curso indicado y existen múltiples secciones.'
        else:
            suggested_nrc = course_matches['NRC'].values[0]
            obs += 'NRC inválido (sección única). '
        academic_unit = course_matches['Escuela'].values[0]
    else:
        academic_unit = course_match_nrc['Escuela'].values[0]
    
    return obs, suggested_nrc, academic_unit

def verify_sigla(df, selected_sigla_format, course_matches, obs, suggested_nrc, selected_nrc, selected_course_name_format):
    suggested_sigla = ''
    if selected_sigla_format not in course_matches['Sigla (formateado)'].values:
        course_match = df.loc[(df['Nombre Curso (formateado)'] == selected_course_name_format) & (df['NRC'].isin([selected_nrc, suggested_nrc]))]
        if len(course_match):
            suggested_sigla = course_match['Sigla'].values[0]
            obs += 'Sigla inválida. '
    
    return obs, suggested_sigla

def verify_name(df, selected_nrc, selected_sigla_format, obs, vrai_check):
    suggested_course_name, academic_unit, in_catalogue, suggested_nrc = '', '', '', ''
    course_matches = df.loc[df['Sigla (formateado)'] == selected_sigla_format]
    if len(course_matches) >= 1:
        suggested_course_name, academic_unit = course_matches['Nombre Curso'].values[0], course_matches['Escuela'].values[0]
        obs += 'Nombre inválido '
        if vrai_check:
            in_catalogue = True
        else:
            in_catalogue = False
        
        if len(course_matches) == 1:
            suggested_nrc = course_matches['NRC'].values[0]
            if suggested_nrc == selected_nrc:
                obs += '(Sección única y NRC correcto). '
                suggested_nrc = ''
            else:
                obs += '(Sección única y NRC incorrecto). '

        else:
            obs += '(Múltiples secciones). '


    else:
        if vrai_check:
            in_catalogue = False
        else:
            obs += 'ERROR, se requiere revisión. No existe un nombre para la sigla indicada. '
    
    return obs, suggested_course_name, academic_unit, in_catalogue, suggested_nrc

def verify_nrc_and_sigla(df, course_matches, selected_course_name_format, selected_sigla_format, selected_nrc, suggested_nrc, obs):
    academic_unit = ''
    obs, suggested_nrc, academic_unit = verify_nrc(df, course_matches, selected_course_name_format, selected_nrc, obs)
    obs, suggested_sigla = verify_sigla(df, selected_sigla_format, course_matches, obs, suggested_nrc, selected_nrc, selected_course_name_format)

    return obs, suggested_nrc, suggested_sigla, academic_unit

A continuación se encuentra el flujo principal de la automatización

In [10]:
def verify_course_info(df, dpt):
    columns = ['NOMBRE COMPLETO', 'RUT UC', 'NOMBRE CURSO INDICADO', 'NOMBRE SUGERIDO', 'Escuela',
                'NRC INDICADO', 'NRC SUGERIDO',  'SIGLA INDICADA', 'SIGLA SUGERIDA', 'EN CATÁLOGO VRAI', 'OBSERVACIONES']
    df_output = pd.DataFrame(columns=columns)

    for index, row in df.iterrows():
        selected_course_name, selected_course_name_format = row['Nombre Curso'], row['Nombre Curso (formateado)']
        selected_sigla, selected_sigla_format, selected_nrc = row['Sigla'], row['Sigla (formateado)'], row['NRC']
        student_name, rut_uc = row['Nombre completo'], row['RUT UC']
        suggested_course_name, suggested_nrc, suggested_sigla, academic_unit, obs, in_vrai_catalogue = '', '', '', '', '', ''

        # Se comienza chequeando si el nombre del curso coincide con alguno del catálogo
        course_matches = df_courses_vrai.loc[df_courses_vrai['Nombre Curso (formateado)'] == selected_course_name_format]

        # En caso de que el chequeo de nombre arroje coincidencias, se procede a verificiar NRC y/o sigla
        if len(course_matches):
            academic_unit = course_matches['Escuela'].values[0]
            in_vrai_catalogue = True
            obs, suggested_nrc, suggested_sigla, academic_unit = verify_nrc_and_sigla(df_courses_vrai, course_matches, 
                                                                                selected_course_name_format, selected_sigla_format, 
                                                                                selected_nrc, suggested_nrc, obs)
            
        # En caso de que el chequeo de nombre no arroje coincidencias, se procede a verificiar el nombre del curso
        else:
            obs, suggested_course_name, academic_unit, in_vrai_catalogue, suggested_nrc = verify_name(df_courses_vrai, selected_nrc, selected_sigla_format, obs, True)
            if not in_vrai_catalogue:
                course_matches = df_courses_global.loc[df_courses_global['Nombre Curso (formateado)'] == selected_course_name_format]
                if len(course_matches):
                    academic_unit = course_matches['Escuela'].values[0]
                    obs, suggested_nrc, suggested_sigla, academic_unit = verify_nrc_and_sigla(df_courses_global, course_matches, selected_course_name_format, 
                                                                                        selected_sigla_format, selected_nrc, suggested_nrc, obs)
                else:
                    obs, suggested_course_name, academic_unit, in_vrai_catalogue, suggested_nrc = verify_name(df_courses_global, selected_nrc, selected_sigla_format, obs, False)
            
        df_output.loc[len(df_output)] = [student_name, rut_uc, selected_course_name, suggested_course_name, 
                                        academic_unit, selected_nrc, suggested_nrc, selected_sigla, suggested_sigla, in_vrai_catalogue, obs]

    if dpt:
        df_output.to_csv('output_dpt.csv', index=False)
    else:
        df_output.to_csv('output_rezagados.csv', index=False)

In [11]:
verify_course_info(df_students, dpt=False)

# Descomentar la línea siguiente si se desea hacer la revisión para cursos deportivos
#verify_course_info(df_students_dpt, dpt=True)